In [1]:
import os
import sys
import time

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
from grouping import *

In [3]:
import pennylane as qml
from pennylane import numpy as np
from measurement_memory import evaluate_eigenstate, evaluate_eigenstate_MM,\
                               GradientDescent, str_to_Pauli
from measurement_memory import get_measurement_list, measurement_rotation


def read_Hamiltonian(mol:str, grouping_type:str):
    path = os.path.abspath(os.path.join("{}_{}_Hamiltonian.txt".format(mol, grouping_type)))
    Hg = []
    Groups = []
    with open(path) as f:
        qubits, terms, groups = f.readline()[:-1].split(' ')
        
        group_ops = []
        group_coeffs = []
        T = 0
        G = 0
        for line in f.readlines():
            s = line.split(' ')
            if s[0] == '\n' or s[0] == None:
                Groups.append(group_ops)
                Hg.append(qml.Hamiltonian(group_coeffs, str_to_Pauli(group_ops), grouping_type='commuting'))
                group_ops = []
                group_coeffs = []
                G += 1
            else:
                group_ops.append(s[0])
                if s[1][-1] == '\n': s[1] = s[1][:-1]
                group_coeffs.append(float(s[1]))
                T += 1
        assert T == int(terms) and G == int(groups), "Fail to read Hamiltonian correctly. Check if the last group end with '\n'."
    return Hg, Groups

In [4]:
Hg, Groups = read_Hamiltonian("H4", "GC")

In [5]:
new_H, new_coeffs, T, Q = get_measurement_basis(Groups[1])

In [10]:
M = is_QWC(new_H, return_basis=True)
print(M)

['X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z']


In [6]:
new_H

['IIIIIIZZ',
 'IIXXZIZZ',
 'IXXXIZZI',
 'IXXIIZZI',
 'IIXIZZII',
 'IIXXZZII',
 'XXXIIZII',
 'XXXXIZII',
 'IXXXZIZZ',
 'IXIIZIIZ',
 'IIIIZIIZ',
 'IXIIIIZZ',
 'IIIIZZZI',
 'IIIXZZZI',
 'XIIXIIIZ',
 'XXIXIIIZ',
 'IIIXIIII',
 'XXIXZIII',
 'XIIXIIZI',
 'IIXXIIZI',
 'IIXXIZIZ',
 'IXXXZIII',
 'IXXIIZIZ',
 'IXIIIIII']

In [7]:
new_coeffs

[1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [8]:
T

['ZIIIZZIZ',
 'IIIZIZII',
 'IZIIIIIZ',
 'IIZIIZZZ',
 'XIIIXIII',
 'XIXXIXZZ',
 'IIXIIZXZ',
 'XXXIIZZX']

In [9]:
Q

['XIIIIIII',
 'IIIXIIII',
 'IXIIIIII',
 'IIXIIIII',
 'IIIIZIII',
 'IIIIIZII',
 'IIIIIIZI',
 'IIIIIIIZ']

array([0., 0., 0., 1., 1., 1., 0., 0.])

16

In [9]:
S = []
for obs in Groups[1]:
    S.append(sympletic_subspace_encoding(obs))
#E = remove_zero(RREF_binary(np.array(S)))
E = np.array(S)
print(len(E))
print(E)

24
[[1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1]
 [0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 0]
 [1 0 0 1 0 1 1 0 0 1 1 1 0 1 0 0]
 [1 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0]
 [0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0]
 [1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0]
 [1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 0]
 [0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
 [0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 1]
 [0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0]
 [1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0]
 [1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1]
 [1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0]
 [0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 0]
 [0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0]
 [1 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]]


[0, 4, 5]

In [13]:
Mols = ["H2", "H4", "H6", "LiH", "H2O", "O2"]
Q = [4, 8, 12, 12, 14, 20]

In [14]:
for i,mol in enumerate(Mols): 
    Hg, Groups = read_Hamiltonian(mol, "GC")
    Tau = Find_Tau(Groups)
    print("qubits: {}".format(Q[i]))
    for tau in Tau:
        print(len(tau))
        print(tau)
    print("--------")

qubits: 4
1
['ZZZZ']
3
['XXYY', 'ZIZI', 'IZIZ']
--------
qubits: 8
7
['XIIIIYYX', 'IXZZZXIZ', 'IIXZIZXI', 'IIIYZIZY', 'IIIIXXYY', 'ZIZIZIZI', 'IZIZIZIZ']
7
['XIZIXZZZ', 'IXZIXZXX', 'IIYIIIYI', 'IIIXXYYI', 'ZIZIZIZI', 'IZIIIIIZ', 'IIIZIZII']
7
['XIIIIXYY', 'IXZIZXII', 'IIXZIZXI', 'IIIXZIZX', 'IIIIXYYX', 'ZIZIZIZI', 'IZIZIZIZ']
7
['XIZIZXYY', 'IXZXIXZX', 'IIYZIIYZ', 'IIIIYXXY', 'ZIZIZIZI', 'IZIZIIII', 'IIIIIZIZ']
8
['XIIIXZII', 'IXIIZXII', 'ZIIIZIII', 'IZIIIZII', 'IIZIIIII', 'IIIZIIII', 'IIIIIIZI', 'IIIIIIIZ']
1
['ZZZZZZZZ']
8
['XIIIXIIZ', 'IIIXZIIX', 'ZIIIZIII', 'IZIIIIII', 'IIZIIIII', 'IIIZIIIZ', 'IIIIIZII', 'IIIIIIZI']
8
['IXIIIXZZ', 'IIXIIZXZ', 'IIIXIZZX', 'ZIIIIIII', 'IZIIIZII', 'IIZIIIZI', 'IIIZIIIZ', 'IIIIZIII']
7
['XIIIXIZZ', 'IIXIZIXZ', 'ZIIIZIII', 'IZIIIIIZ', 'IIZIIIZI', 'IIIZIIIZ', 'IIIIIZIZ']
--------
qubits: 12
11
['XIZIIIIIZXYY', 'IXZIIIIIZXII', 'IIYIIIIZIZYI', 'IIIXIXZIIXZX', 'IIIIXZXIIXZX', 'IIIIIIIXZIZX', 'IIIIIIIIYYXX', 'ZIZIIIIIZIZI', 'IZIIIIIZIZIZ', 'IIIZIZIIIIII', 'I

qubits: 14
14
['XIIIIIXIIIZZII', 'IXIIXXZIIIXZII', 'IIXXIIIIIIIIXX', 'IIIIIIZXIIZXII', 'ZIIIIIZIIIIIII', 'IZIIIIIIIIZIII', 'IIZIIIIIIIIIIZ', 'IIIZIIIIIIIIIZ', 'IIIIZIIIIIZIII', 'IIIIIZIIIIZIII', 'IIIIIIIZIIIZII', 'IIIIIIIIZIIIII', 'IIIIIIIIIZIIII', 'IIIIIIIIIIIIZZ']
14
['XIIIIIIIIIZZXZ', 'IIXXIIIIIIIYYI', 'IIIIXIXIIIIIIZ', 'IIIIIXZIIIZZZX', 'IIIIIIIIIIYIYI', 'ZIIIIIIIIIZZZI', 'IZIIIIIIIIIIII', 'IIZIIIIIIIIZII', 'IIIZIIIIIIIZII', 'IIIIZIZIIIIIII', 'IIIIIZIIIIIIIZ', 'IIIIIIIZIIIIII', 'IIIIIIIIZIIIII', 'IIIIIIIIIZIIII']
14
['XIIIIIIIIIIIXZ', 'IIXIIIIIIIZIXZ', 'IIIXIIIIIIZXIZ', 'IIIIIXIIIIZZZX', 'IIZIIIIIIIXZXI', 'ZIZIIIIIIIZIZI', 'IZIIIIIIIIIIII', 'IIIZIIIIIIIZII', 'IIIIZIIIIIIIII', 'IIIIIZIIIIIIIZ', 'IIIIIIZIIIIIII', 'IIIIIIIZIIIIII', 'IIIIIIIIZIIIII', 'IIIIIIIIIZIIII']
12
['XIIIIXYIZZIIIY', 'IXIIIIIIZZIXII', 'IIXIIIXZIIIIIZ', 'IIIXIIZXIIIIIZ', 'IIIIXIXZIIIIIZ', 'IIIIIIIIIIXIXI', 'ZIIIIIIIIIIIIZ', 'IZIIIIIIIIIZII', 'IIZIZIZIIIIIIZ', 'IIIZIIIZIIIIII', 'IIIIIZIIIIIIIZ', 'IIIIIIIIIIZIZI']
1

qubits: 20
20
['XXIIIIIIIIIIIIIIIIYY', 'IIXIIIIIIIIIXZZZZZZZ', 'IIIXIIIIIIIIZIIIIIZX', 'IIIIXIIIIIXZIIZZZZII', 'IIIIIXIIIIZXIIZZZZII', 'IIIIIIXIIIZZZZXZZZII', 'IIIIIIIXIIZZZZZXZZII', 'IIIIIIIIXIZZZZZZXZII', 'IIIIIIIIIXZZZZZZZXII', 'IIIIIIIIIIIIIXZZZZZX', 'ZIIIIIIIIIIIIIIIIIZI', 'IZIIIIIIIIIIIIIIIIZI', 'IIZIIIIIIIIIZIIIIIII', 'IIIZIIIIIIIIIZIIIIZZ', 'IIIIZIIIIIZIIIIIIIII', 'IIIIIZIIIIIZIIIIIIII', 'IIIIIIZIIIIIIIZIIIII', 'IIIIIIIZIIIIIIIZIIII', 'IIIIIIIIZIIIIIIIZIII', 'IIIIIIIIIZIIIIIIIZII']
20
['XIXIIIIIIIIIIIIIIIII', 'IXIXIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIXXIIIIXX', 'ZIZIIIIIIIIIIIIIIIII', 'IZIZIIIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIIII', 'IIIIIIIZIIIIIIIIIIII', 'IIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIZIIIIIIZ', 'IIIIIIIIIIIIIZIIIIIZ', 'IIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIZZ']
19
['XIIIIIIIIIIIIXZZ

In [20]:
Hg, Groups = read_Hamiltonian("O2", "GC")

In [78]:
def single_qubit_form(self):
        '''
        Returns
        ----------
        hamiltonian : BinaryHamiltonian
            The original hamiltonian in qubit-wise commuting form
        lagrangian_basis : list of BinaryPauliStrings 
            Represents the basis of original Hamiltonian
        new_basis : list of BinaryPauliStrings 
            Represents the basis of new Hamiltonian
        '''
        lagrangian_basis = get_lagrangian_subspace(self.get_binary())
        new_basis = self.get_single_qubit_basis(lagrangian_basis)
        lagrangian_basis = [BinaryPauliString(p) for p in lagrangian_basis]
        new_basis = [BinaryPauliString(p) for p in new_basis]
        qubit_wise_hamiltonian = self.basis_transform(lagrangian_basis,
                                                      new_basis)
        return qubit_wise_hamiltonian, lagrangian_basis, new_basis

'IIZZ'

In [91]:
def is_QWC(P:list):
    # Input : List of Pauli obsevables
    # OutPut : True/False
    q = len(P[0])
    B = ['I']*q
    for i in range(q):
        for obs in P:
            if obs[i] != 'I':
                if B[i] == 'I': B[i] = obs[i]
                if B[i] != obs[i]: return False
    return True